In [1]:
"""
콜백은 모델을 훈련할 때 보조적인 작업을 수행할 수 있도록 도와주는 객체
모델을 훈련할 때 사용하는 fit()메소드에 callbacks 매개변수로 지정할 수 있다.
tensorflow,keras,callbacks 패키지 내 다양한 콜백이 정의되어 있다
이중 가장 많이 활용하는 콜백 함수 몇가지를 소개
"""
import tensorflow as tf

#ㅔ라스의 내장 데이터셋에서 mnist 데이터셋 로드
mnist = tf.keras.datasets.mnist

In [2]:
#load_data()로 데이터셋 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 정규화
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# 모델정의 
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    # 노드=10개가 되어야 한다
    tf.keras.layers.Dense(10, activation="softmax"),
])

In [3]:
#모델 컴파일
model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics = ["accuracy"])

In [4]:
"""
#모델 체크 포인트
모델 체크포인트는 가장 많이 활용되는 콜백이다. 모델 훈련 시 콜백으로 지정해 줄 수 있으며
epoch별 모델의 가중치를 저장한다. 저장할 때 마치 체크포인트를 생성하듯 미리 정해 놓은 규칙에
의하여 체크포인트를 생성하고 저장한다

주요 하이퍼파라미터에 대한 세부 내용은 다음과 같다

filepath : 체크포인트의 저장 경로 지정
save_weights_only : 가중치만 저장할지 여부 설정
save_best_only : monitor 기준으로 가장 높은 epoch만 저장할지 아니면 매 epoch별 저장할지 여부를 결정
monitor : 저장시 기준이 되는 지표 설정 "val_loss"로 지정시 검증 손실이 가장 낮은 epoch 의 가중치를 저장
verbose : 1로 설정 시 매 epoch 별 저장 여부를 알려주는 로그 메세지 출력
"""
# 체크포인트 설정
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = "tmp_checkpoint.ckpt",
                                               save_weights_only = True,
                                               save_best_only = True,
                                               monitor = "val_loss",
                                               verbose=1)

In [5]:
"""
모델 체크포인트 객체를 생성후 모델 훈련시 callbacks 매개변수에 지정하면 된다
"""
model.fit(x_train, y_train,
         validation_data=(x_test,y_test),
         epochs=10,
         callbacks=[checkpoint]
         )

Epoch 1/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.2314 - accuracy: 0.9303 ETA: 0s - loss:
Epoch 00001: val_loss improved from inf to 0.10790, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2314 - accuracy: 0.9303 - val_loss: 0.1079 - val_accuracy: 0.9666
Epoch 2/10
1864/1875 [============================>.] - ETA: 0s - loss: 0.0930 - accuracy: 0.9715
Epoch 00002: val_loss improved from 0.10790 to 0.08653, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0931 - accuracy: 0.9715 - val_loss: 0.0865 - val_accuracy: 0.9735
Epoch 3/10
1866/1875 [============================>.] - ETA: 0s - loss: 0.0640 - accuracy: 0.9792
Epoch 00003: val_loss improved from 0.08653 to 0.07534, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0638 - accuracy: 0.9793 - val_loss: 0.0753 - val_accuracy: 0.9762
Epoch 4/10
1865/1

In [6]:
"""
#모델 체크포인트
매 epoch 마다 모델 체크포인트의 저장 여부를 알려주는 로그 메세지가 출력되며,
가장 검증 손실이 낮았던 epoch일 때의 가중치가 현재 모델 체크포인트에
저장되었음을 알 수 있다

저장한 가중치를 model 인스턴스에 적용하려면, load_weights()메소드에 모델 체크포인트 파일
경로를 지정하여 호출해 주어야 한다 이처럼 모델에 저장한 가중치를 명시적으로 로드해주어야
검증 손실이 가장 낮았던 가중치가 모델에 로드된다. 이렇게 명시적으로 호출하지 않으면 
훈련이 완료되더라고 가중치가 로드되지 않음에 유의한다
"""
# 모델 체크포인트 로드전 
loss, acc =model.evaluate(x_test, y_test)
print(f"체크포인트 로드전 : loss: {loss:3f}, acc{acc:3f}")

#체크포인트 파일을 모델에 로드
model.load_weights("tmp_checkpoint.ckpt")
loss, acc = model.evaluate(x_test, y_test)
print(f"체크포인트 로드후 : loss: {loss:3f}, acc{acc:3f}")


313/313 [==============================] - 1s 4ms/step - loss: 0.0925 - accuracy: 0.9783
체크포인트 로드전 : loss: 0.092537, acc0.978300
313/313 [==============================] - 1s 4ms/step - loss: 0.0695 - accuracy: 0.9787
체크포인트 로드후 : loss: 0.069532, acc0.978700


In [8]:
"""
# 조기 종료

tensorflow.keras.callbacks.EarlyStopping() 객체로 생성하며 모델 훈련 시 patience에 
지정된 epoch 안에 손실이 줄어들지 않는다면 모델 훈련은 조기 종료한다
다음은 조기 종료 기준으로 되는 지표를 검증 손실로 설정하고, patience =3 으로 설정해 
조기 종료 콜백을 생성한다 
따라서 3 epoch 동안 손실이 줄어들지 않으면 모델 훈련이 종료된다
"""
# 모델 인스턴스 생성

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    # 노드=10개가 되어야 한다
    tf.keras.layers.Dense(10, activation="softmax"),
])

#모델 컴파일
model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics = ["accuracy"])

# EarlyStopping 콜백 생성
earlystopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [9]:
"""
모델 체크포인트 콜백과 마찬가지로, 조기 종료 콜백 또한 모델을 훈련하는 fit()메소드 안에
callbacks 매개변수로 지정한다
"""
model.fit(x_train, y_train,
         validation_data = (x_test, y_test),
         epochs=20,
         callbacks=[earlystopping])

Epoch 1/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2315 - accuracy: 0.9304 - val_loss: 0.1204 - val_accuracy: 0.9646
Epoch 2/20
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0954 - accuracy: 0.9711 - val_loss: 0.0854 - val_accuracy: 0.9740
Epoch 3/20
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0648 - accuracy: 0.9797 - val_loss: 0.0757 - val_accuracy: 0.9765
Epoch 4/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0498 - accuracy: 0.9839 - val_loss: 0.0715 - val_accuracy: 0.9800
Epoch 5/20
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0391 - accuracy: 0.9871 - val_loss: 0.0800 - val_accuracy: 0.9776
Epoch 6/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0326 - accuracy: 0.9893 - val_loss: 0.0761 - val_accuracy: 0.9772
Epoch 7/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0284 - accuracy: 0.9910 - val_loss: 0.0775 - val_accurac

In [ ]:
"""
val_loss 의 변화를 기준으로, 조기종료 콜백이 작동 
조기종료 콜백에서 검증 손실 기준 3 epoch 동안 개선이 일어나지 않으면 조기 종료하는 규칙에
걸려 훈련을 조기 종료 했다
"""

In [14]:
"""
# 학습률 스케줄러 
tensorflow.keras.callback.LearningRateScheduler() 객체로 생성하여 훈련에 대한 학습률 조정한다.
학습률을 특징 로직에 의하여 제어하고자 할 때 로직을 함수로 구현한 뒤 LearningRateScheduler 객체에 적용해 주면 된다

다음은 5 epoch 동안 학습률을 유지하되 6 epoch부터는 학습률을 점차 감소시키는 예
"""
def scheduler(epoch, lr):
    tf.print(f"learning_rate:{lr:5f}")
    #첫 5 epoch동안 유지
    if epoch < 5 :
        return lr
    else:
        #학습률 감소 적용
        return lr * tf.math.exp(-0.1)
    
    #콜백 객체 생성 및 scheduler 함수 적용
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [15]:
"""모델 인스턴스를 다시 생성하고 컴파일
초기 학습률을 출력하여 확인 """
# 모델 인스턴스 생성

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    # 노드=10개가 되어야 한다
    tf.keras.layers.Dense(10, activation="softmax"),
])

#모델 컴파일 - 학습률 스케줄러 
model.compile(tf.keras.optimizers.SGD(), loss="sparse_categorical_crossentropy", metrics = ["accuracy"])

#초기 학습률 확인(0.01)
print(round(model.optimizer.lr.numpy(), 5))

0.01


In [16]:
"""
schedulere 함수에 적용하고자 하는 학습률 감소 로직을 구현한뒤 LearningRateScheduler 객체에 적용
"""
model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         epochs=10,
         # 학습률 스케줄러 적용
         callbacks=[lr_scheduler]
         )

#최종 학습률 스케줄러 확인
round(model.optimizer.lr.numpy(), 5)

learning_rate:0.010000
Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.6563 - accuracy: 0.8175 - val_loss: 0.3002 - val_accuracy: 0.9136 - lr: 0.0100
learning_rate:0.010000
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2715 - accuracy: 0.9222 - val_loss: 0.2321 - val_accuracy: 0.9315 - lr: 0.0100
learning_rate:0.010000
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2107 - accuracy: 0.9395 - val_loss: 0.1900 - val_accuracy: 0.9451 - lr: 0.0100
learning_rate:0.010000
Epoch 4/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1725 - accuracy: 0.9506 - val_loss: 0.1714 - val_accuracy: 0.9484 - lr: 0.0100
learning_rate:0.010000
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1457 - accuracy: 0.9571 - val_loss: 0.1416 - val_accuracy: 0.9571 - lr: 0.0100
learning_rate:0.010000
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss

0.00607

In [ ]:
"""
학습률이 감소하여 10epoch 훈련이 완료된 후에는 최종 학습률이 0.00607로 감소한 것을 확인 할 수 있다
scheduler 함수에 구현한 로직을 수정하여 학습률 감소 크기나 시점을 다르게 하여 적용할 수 있다

"""

In [18]:
"""
#텐서보드
텐서보드는 훈련에 대한 시각화를 실시간으로 제공하는 유용한 도구
시각화 라이브러리인 matplotlib이나 seaborn으로 epoch 별 손실과 정확도 등의 평가지표를 
시각화하는 방법으로 History객체를 활용하는 예제를 앞에서 소개한 바 있다
텐서보드를 활용하면 epoch별 훈련 손실 및 평가 지표를 시각화해 차트로 보여주는 것은 물론이고,
모델의 구조를 시각화해 보여주거나 레이어의 가중치 분포도를 시각화로 제공한다
또한 모델 훈련시 시각화 차트를 실시간으로 업데이트해 제공하는 기능도 포함


텐서보드 시각화를 위해서는 tensorflow.keras.callbacks.TesorBoard() 객체로 생성하여
콜백 매개변수에 적용하면 된다. 가중치 초기화를 위해 모델 인스턴스를 다시 만들어 주는 점에 유의
"""

# 모델 인스턴스 생성

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    # 노드=10개가 되어야 한다
    tf.keras.layers.Dense(10, activation="softmax"),
])

#모델 컴파일 - 학습률 스케줄러 
model.compile(tf.keras.optimizers.SGD(), loss="sparse_categorical_crossentropy", metrics = ["accuracy"])


#텐서보드 저장 경로 지정
log_dir = "tensorboard"

#텐서보드 콜백 정의
tensorboard = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1)

model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         epochs=10,
         callbacks =[tensorboard],)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6318 - accuracy: 0.8273 - val_loss: 0.2952 - val_accuracy: 0.9131
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2630 - accuracy: 0.9247 - val_loss: 0.2331 - val_accuracy: 0.9300
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2042 - accuracy: 0.9413 - val_loss: 0.1825 - val_accuracy: 0.9480
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1669 - accuracy: 0.9525 - val_loss: 0.1549 - val_accuracy: 0.9566
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1402 - accuracy: 0.9599 - val_loss: 0.1395 - val_accuracy: 0.9610
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1204 - accuracy: 0.9654 - val_loss: 0.1190 - val_accuracy: 0.9651
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1046 - accuracy: 0.9700 - val_loss: 0.1104 - val_accur

In [19]:
"""
훈련이 완료된 후 노트북의 매직 커멘드를 입력하여 텐서보드를 바로 출력
"""
#텐서보드 extension로드
%load_ext tensorboard

#텐서보드 출력 매직 커멘드
%tensorboard --logdir {log_dir}